In [1]:
###Oscilloscope Readout#####
import pyvisa
import matplotlib.pyplot as plt
import time
import numpy as np
from IPython import display
%matplotlib qt5

In [2]:
rm=pyvisa.ResourceManager()
scope= rm.open_resource('USB0::0x0957::0x179A::MY52491615::INSTR') #Establish communication with the Oscilloscope

In [3]:
#Establish settings for the Oscilloscope

# vRange=1 refers to 125mv/div. vRange=0 refers to smallest possible value (10mV/div)
vRange=3 #code works for max 16 which refers to 2V/div
tRange= 500e-9 #refers to full scope i.e s/div=tRange/10
trigLevel=0
ch=1

In [4]:
scope.write('*rst') #Check that Oscilloscope is responding
scope.query('*opc?')

'1\n'

In [5]:
scope.write(f'channel{ch}:range {vRange}') #Change settings in Oscilloscope according to the settings above
scope.write(f'timebase:range {tRange}')
scope.write('trigger:mode edge')
scope.write(f'trigger:level channel{ch}, {trigLevel}')

27

In [6]:
scope.write(f'waveform:source channel{ch}') #Establish format of data from oscilloscope
scope.write('waveform:format byte')
scope.write('digitize')
data= scope.query_binary_values('waveform:data?', datatype='b')

In [7]:
xIncrement = float(scope.query('waveform:xincrement?')) #Query all needed parameters from oscilloscope
xOrigin = float(scope.query('waveform:xorigin?'))
xref= float(scope.query('waveform:xreference?'))
yIncrement = float(scope.query('waveform:yincrement?'))
yOrigin = float(scope.query('waveform:yorigin?'))
yref= float(scope.query('waveform:yreference?'))
length = len(data)

In [8]:
#time= []
#wfm =[]
#for t in range(length):
#    time.append((t*xIncrement)+ xOrigin)
#for d in data:
#    wfm.append((d*yIncrement)+ yOrigin)

#Convert data to seconds and Volts
time=[(t* xIncrement) +xOrigin for t in range(length)]
wfm=[((d+128)*yIncrement)+ yOrigin for d in data]




128.0


In [9]:
#Plot Data
plt.plot(time,wfm)